In [1]:

!pip install opencv-python-headless


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import normalize, to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D
from keras.layers import  MaxPooling2D
from keras.layers import  Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense

In [3]:
image_directory = 'C:\\Users\\Rasagna\\Downloads\\Brain CNN'

no_tumor_images=os.listdir(image_directory+ '/no')
yes_tumor_images=os.listdir(image_directory+ '/yes')

print('No Tumor: ', len(no_tumor_images))
print('Tumor: ',len(yes_tumor_images))

No Tumor:  1500
Tumor:  1500


In [4]:
dataset = []
label = []
INPUT_SIZE = 128

In [5]:
c=1
for i, image_name in enumerate(no_tumor_images):
    if image_name.split('.')[-1].lower() == 'jpg':
        image_path = os.path.join(image_directory, 'no', image_name)
        if(c==1):
            print(image_path)
            c=10
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue  # Skip to the next image

        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)  # Assuming 0 represents images without tumors

C:\Users\Rasagna\Downloads\Brain CNN\no\no0.jpg


In [6]:
c=10
for i,image_name in enumerate(yes_tumor_images):
    if  image_name.split('.')[-1].lower() == 'jpg':
        image_path = os.path.join(image_directory, 'yes', image_name)
        if(c==1):
            print(image_path)
            c-=1
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Error loading image: {image_path}")
            continue  # Skip to the next image

        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [7]:
print(len(dataset))
print(len(label))

3000
3000


In [8]:
dataset=np.array(dataset)
label=np.array(label)

In [9]:
x_train,x_test,y_train,y_test= train_test_split(dataset, label, test_size = 0.2, random_state = 42)
print("train test split completed")

train test split completed


In [10]:
print(x_train.shape) 
print(y_train.shape) 
print(x_test.shape)
print(y_test.shape)

(2400, 128, 128, 3)
(2400,)
(600, 128, 128, 3)
(600,)


In [11]:
x_train = normalize( x_train, axis =1)
x_test = normalize( x_test, axis =1)

In [12]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
#model.add(Dense(2))
model.add(Activation('sigmoid'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                                                 
 activation (Activation)     (None, 126, 126, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 activation_1 (Activation)   (None, 61, 61, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                   

In [15]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
model.fit(x_train, y_train, 
batch_size = 16, 
verbose = 1, epochs = 9, 
validation_data = (x_test, y_test),
shuffle = 'False')

Epoch 1/9


150/150 [==============================] - 44s 268ms/step - loss: 0.5185 - accuracy: 0.7475 - val_loss: 0.3814 - val_accuracy: 0.8383
Epoch 2/9
150/150 [==============================] - 38s 254ms/step - loss: 0.3414 - accuracy: 0.8537 - val_loss: 0.2456 - val_accuracy: 0.8950
Epoch 3/9
150/150 [==============================] - 38s 252ms/step - loss: 0.1999 - accuracy: 0.9196 - val_loss: 0.1898 - val_accuracy: 0.9267
Epoch 4/9
150/150 [==============================] - 38s 253ms/step - loss: 0.1165 - accuracy: 0.9621 - val_loss: 0.1097 - val_accuracy: 0.9683
Epoch 5/9
150/150 [==============================] - 38s 256ms/step - loss: 0.0630 - accuracy: 0.9817 - val_loss: 0.1063 - val_accuracy: 0.9717
Epoch 6/9
150/150 [==============================] - 38s 253ms/step - loss: 0.0566 - accuracy: 0.9804 - val_loss: 0.0842 - val_accuracy: 0.9783
Epoch 7/9
150/150 [==============================] - 38s 254ms/step - loss: 0.0325 - accuracy: 0.9892 - val_loss: 0.1387 - val_accurac

In [17]:
model.save('Brain Tumor detection.g9')

INFO:tensorflow:Assets written to: Brain Tumor detection.g9\assets


INFO:tensorflow:Assets written to: Brain Tumor detection.g9\assets


In [18]:
from keras.models import load_model 
model = load_model('Brain Tumor detection.g9')

In [19]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1]*100,"%")

Test loss: 0.12003958225250244
Test accuracy: 98.00000190734863 %


In [21]:
image = cv2.imread(r'C:\Users\Rasagna\Downloads\Brain CNN\no\no0.jpg')
img = Image.fromarray(image)
img = img.resize((128,128))
img = np.array(img)
print(img)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [25]:
input_img = np.expand_dims(img, axis=0)
res = model.predict(input_img)
if(res==[[0.]]):
    print("No tumor")
else:
    print("Tumor Detected")

1/1 [==============================] - 0s 80ms/step
No tumor


In [26]:
image = cv2.imread(r"C:\Users\Rasagna\Downloads\Brain CNN\yes\y982.jpg")
img = Image.fromarray(image)
img = img.resize((128,128))
img = np.array(img)
print(img)

[[[90 90 90]
  [77 77 77]
  [78 78 78]
  ...
  [26 26 26]
  [30 30 30]
  [28 28 28]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [45 45 45]
  [30 30 30]
  [27 27 27]]

 [[ 1  1  1]
  [ 1  1  1]
  [ 2  2  2]
  ...
  [40 40 40]
  [28 28 28]
  [27 27 27]]

 ...

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [40 40 40]
  [28 28 28]
  [28 28 28]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [39 39 39]
  [28 28 28]
  [28 28 28]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [34 34 34]
  [31 31 31]
  [30 30 30]]]


In [27]:
input_img = np.expand_dims(img, axis=0)
res = model.predict(input_img)
if(res==[[0.]]):
    print("No tumor")
else:
    print("Tumor Detected")

1/1 [==============================] - 0s 80ms/step
Tumor Detected
